In [1]:
import ee
import folium
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from geopy.geocoders import Nominatim
from scipy.ndimage import label
from skimage.morphology import remove_small_objects

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
STANDARD_DENSITIES = {
    'urban': 160,    
    'suburban': 275, 
    'rural': 400     
}

In [4]:
def add_ee_layer(self, ee_object, vis_params, name):
    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

def classify_pincode(pincode):
    """Classify pincode as urban/suburban/rural based on first digit"""
    first_digit = pincode[0]
    if first_digit in ['1', '2', '3']:  # Metro cities
        return 'urban'
    elif first_digit in ['4', '5', '6']:  # Tier-2 cities
        return 'suburban'
    else:  # Villages/small towns
        return 'rural'

def pincode_to_coords(pincode):
    """Convert pincode to coordinates with urban/rural classification"""
    geolocator = Nominatim(user_agent="tree_density_app")
    try:
        location = geolocator.geocode(f"{pincode}, India")
        if location:
            area_type = classify_pincode(pincode)
            return location.latitude, location.longitude, area_type
        raise ValueError("Pincode not found")
    except Exception as e:
        print(f"Error: {e}. Using fallback coordinates (Patna).")
        return 25.5941, 85.1376, 'suburban'  # Default fallback

def compute_ndvi(image):
    """Calculate NDVI from Sentinel-2 imagery"""
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

def create_grids(roi, grid_size_km=1):
    """Create 1km x 1km grids within ROI"""
    bounds = roi.bounds().getInfo()['coordinates'][0]
    min_long, min_lat = bounds[0]
    max_long, max_lat = bounds[2]
    
    grid_size_deg = 0.009  # ~1km in degrees
    grids = []
    
    current_lat = min_lat
    while current_lat < max_lat:
        current_long = min_long
        while current_long < max_long:
            grid = ee.Geometry.Rectangle(
                [current_long, current_lat, 
                 current_long + grid_size_deg, 
                 current_lat + grid_size_deg],
                None, False
            )
            grids.append(grid)
            current_long += grid_size_deg
        current_lat += grid_size_deg
    return grids

def build_unet_model(input_shape=(256, 256, 6)):
    """U-Net model for tree canopy segmentation"""
    inputs = tf.keras.Input(shape=input_shape)
    
    # Encoder
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    # Decoder
    u3 = layers.UpSampling2D((2, 2))(p2)
    c3 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
    c3 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c3)
    
    u4 = layers.UpSampling2D((2, 2))(c3)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u4)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def count_trees(prediction):
    """Count trees from U-Net prediction mask"""
    binary_mask = (prediction > 0.5).astype(np.uint8)
    labeled, num_features = label(binary_mask)
    cleaned = remove_small_objects(labeled, min_size=5)
    return np.max(cleaned) if np.max(cleaned) > 0 else 0


In [5]:
def main():
    print("\n🌳 Tree Density Analyzer 🌳")
    print("="*30)
    
    # Input method selection
    print("\nChoose input method:")
    print("1. Pincode (6-digit Indian)")
    print("2. Manual Coordinates")
    choice = input("Enter choice (1/2): ").strip()
    
    if choice == '1':
        pincode = input("Enter pincode: ").strip()
        if len(pincode) != 6 or not pincode.isdigit():
            print("Invalid pincode format. Using Patna as fallback.")
            lat, lon, area_type = 25.5941, 85.1376, 'suburban'
        else:
            lat, lon, area_type = pincode_to_coords(pincode)
        
        std_density = STANDARD_DENSITIES[area_type]
        buffer_deg = 0.02  # ~2km radius
        roi = ee.Geometry.Rectangle([
            lon - buffer_deg, lat - buffer_deg,
            lon + buffer_deg, lat + buffer_deg
        ])
        print(f"\n Analyzing {area_type} area having pincode {pincode}")
        
    elif choice == '2':
        print("\nEnter coordinates:")
        min_long = float(input("Min Longitude: "))
        min_lat = float(input("Min Latitude: "))
        max_long = float(input("Max Longitude: "))
        max_lat = float(input("Max Latitude: "))
        roi = ee.Geometry.Rectangle([min_long, min_lat, max_long, max_lat])
        lat, lon = (min_lat + max_lat)/2, (min_long + max_long)/2
        std_density = 200  # Default for manual input
        print("\n Analyzing custom coordinate area")
    
    else:
        print("Invalid choice")
        return
    
    print(f" Standard density: {std_density} trees/sq km")
    
    # Load and process satellite data
    print("\n Loading Sentinel-2 data...")
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(roi) \
        .filterDate('2024-01-01', '2024-12-31') \
        .median() \
        .clip(roi)
    
    # Compute NDVI
    ndvi = compute_ndvi(sentinel)
    
    # Create analysis grids
    grids = create_grids(roi)[:5]  
    
    # Initialize U-Net model (pretrained weights would be loaded here)
    print("\n Initializing tree detection model...")
    model = build_unet_model()
    
    # Analyze each grid
    print("\n Analyzing tree cover...")
    tree_counts = []
    
    m = folium.Map(location=[lat, lon], zoom_start=14)
    
    for i, grid in enumerate(grids):
        # In production: Export grid image -> predict with U-Net -> count trees
        # For demo: Simulate prediction
        simulated_trees = np.random.randint(std_density-50, std_density+50)
        tree_counts.append(simulated_trees)
        
        # Add grid to map
        folium.GeoJson(
            grid.getInfo(),
            style_function=lambda x, count=simulated_trees: {
                'color': 'green' if count >= std_density else 'red',
                'weight': 2,
                'fillOpacity': 0.1
            },
            tooltip=f"Grid {i+1}: {simulated_trees} trees/sq km"
        ).add_to(m)
    
    # Calculate statistics
    avg_density = np.mean(tree_counts)
    compliance = " Meets" if avg_density >= std_density else " Below"
    
    # Results
    print("\n Results:")
    print("="*30)
    print(f"Average tree density: {avg_density:.1f} trees/sq km")
    print(f"Standard density: {std_density} trees/sq km")
    print(f"Status: {compliance} green cover standards")
    
    # Visualization
    print("\n Generating map...")
    m.add_ee_layer(ndvi, {
        'min': 0, 
        'max': 1, 
        'palette': ['red', 'yellow', 'green']
    }, 'Vegetation Index')
    m.add_child(folium.LayerControl())
    display(m)

if __name__ == "__main__":
    main()


🌳 Tree Density Analyzer 🌳

Choose input method:
1. Pincode (6-digit Indian)
2. Manual Coordinates

 Analyzing urban area having pincode 201310
 Standard density: 160 trees/sq km

 Loading Sentinel-2 data...

 Initializing tree detection model...

 Analyzing tree cover...

 Results:
Average tree density: 164.2 trees/sq km
Standard density: 160 trees/sq km
Status:  Meets green cover standards

 Generating map...
